In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.special import logit, expit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss

import warnings
warnings.filterwarnings('ignore')
rng = np.random.default_rng(seed = 456)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [3]:
def to_df(X, y):
  df = X.copy()
  df['homeWin'] = y
  return df

def logit(X_train, y_train, X_test, y_test):
  f = y_train.name + ' ~ ' + ' + '.join([col for col in X_train.columns])
  result = smf.logit(f, data=to_df(X_train, y_train)).fit()
  y_pred = result.predict(X_test)
  return brier_score_loss(y_test, y_pred)

def randomForest(X_train, y_train, X_test, y_test):
  rf = RandomForestClassifier()
  rf.fit(X_train, y_train)
  y_pred = rf.predict_proba(X_test)[:, 1]
  return brier_score_loss(y_test, y_pred)

In [4]:
df = pd.read_csv('soccer18.csv', parse_dates = ['Date'])
df = df.replace('Evian Thonon Gaillard', 'Evian')
df['GameID'] = df.index
df['PD_H'] = df.FTHG - df.FTAG
df['PD_A'] = df.FTAG - df.FTHG
df = df.sort_values('Date')

We can add a few metrics based off data we have to add dimensionality to the dataset.

From Q1
* h/a historical goal differential
* absolute disparity




In [5]:
df_melt = pd.melt(df, id_vars='GameID', value_vars=['HomeTeam', 'AwayTeam'], var_name='isHome', value_name='Team')
df_melt['isHome'] = np.where(df_melt.isHome =='HomeTeam', 'H', 'A')
df_melt2 = pd.melt(df, id_vars='GameID', value_vars=['PD_H', 'PD_A'], var_name='isHome', value_name='PD')
df_melt2['isHome'] = np.where(df_melt2.isHome =='PD_H', 'H', 'A')
df_merge = df_melt.merge(df_melt2, on=['GameID', 'isHome']).merge(df[['GameID', 'Date']], on='GameID').sort_values('Date')
df_merge['hAGD'] = df_merge.groupby(['Team']).PD.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df_pivot = df_merge.pivot(index='GameID', columns='isHome')
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df[['GameID', 'Div', 'Y', 'HomeTeam', 'AwayTeam']].merge(
    df_pivot[['GameID', 'hAGD_H', 'hAGD_A']], on='GameID')
df = pd.merge(df, df_pivot[['GameID', 'hAGD_H', 'hAGD_A']], how='left')
df['homeWin'] = 1*(df.FTHG > df.FTAG)

In [6]:
train = df[df.Y < 18]
test = df[df.Y == 18]

Using Question 1d Intercept + historical goal differential

In [7]:
result = smf.logit('homeWin ~ 1+hAGD_H+hAGD_A', data=train).fit()
y_pred = result.predict(test)
brier_score_loss(test['homeWin'], y_pred)

Optimization terminated successfully.
         Current function value: 0.630677
         Iterations 5


0.21726101075298782

Here we have the baseline model with a brier score of **0.21727**

In [8]:
X_train, y_train = df[df.Y < 18][['hAGD_H', 'hAGD_A']], df[df.Y < 18].homeWin
X_test, y_test = df[df.Y == 18][['hAGD_H', 'hAGD_A']], df[df.Y == 18].homeWin

In [9]:
bsl = randomForest(X_train, y_train, X_test, y_test)
print('Random forest brier score:', bsl)

Random forest brier score: 0.2586134234144064


Now we have the baseline model, we can add more features with the other available columns of data (HS/AS, HST/AST, home_xG/away_xG). Since these stats are only calculated during/after the game, we can convert this data into historical features like we did the goal differentials.

In [10]:
df['STD_H'] = df.HST - df.AST
df['STD_A'] = df.AST - df.HST
df['xGD_H'] = df.home_xG - df.away_xG
df['xGD_A'] = df.away_xG - df.home_xG

In [11]:
df_melt3 = pd.melt(df, id_vars='GameID', value_vars=['xGD_H', 'xGD_A'], var_name='isHome', value_name='xGD')
df_melt3['isHome'] = np.where(df_melt3.isHome =='xGD_H', 'H', 'A')
df_merge = df_melt.merge(df_melt3, on=['GameID', 'isHome']).merge(df[['GameID', 'Date', 'Y']], on='GameID').sort_values('Date')
df_merge['hxGD'] = df_merge.groupby(['Team', 'Y']).xGD.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df_pivot = df_merge.pivot(index='GameID', columns='isHome')
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df[['GameID', 'Div', 'Y', 'HomeTeam', 'AwayTeam']].merge(df_pivot[['GameID', 'hxGD_H', 'hxGD_A']], on='GameID')
df = pd.merge(df, df_pivot[['GameID', 'hxGD_H', 'hxGD_A']], how='left')

In [12]:
df_melt4 = pd.melt(df, id_vars='GameID', value_vars=['STD_H', 'STD_A'], var_name='isHome', value_name='STD')
df_melt4['isHome'] = np.where(df_melt4.isHome =='STD_H', 'H', 'A')
df_merge = df_melt.merge(df_melt4, on=['GameID', 'isHome']).merge(df[['GameID', 'Date', 'Y']], on='GameID').sort_values('Date')
df_merge['hST'] = df_merge.groupby(['Team', 'Y']).STD.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df_pivot = df_merge.pivot(index='GameID', columns='isHome')
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df[['GameID', 'Div', 'Y', 'HomeTeam', 'AwayTeam']].merge(df_pivot[['GameID', 'hST_H', 'hST_A']], on='GameID')
df = pd.merge(df, df_pivot[['GameID', 'hST_H', 'hST_A']], how='left')

In [13]:
# Drop all game and post game stats
pgs = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'home_xG', 'away_xG', 'PD_H', 'PD_A', 'STD_H', 'STD_A', 'xGD_H', 'xGD_A']
df_hs = df.drop(pgs, 1)

In [14]:
# Drop GameID and Y after train/test splits
X_train, y_train = df_hs[df.Y < 18].drop('homeWin', 1), df_hs[df.Y < 18].homeWin
X_test, y_test = df_hs[df.Y == 18].drop('homeWin', 1), df_hs[df.Y == 18].homeWin
X_train = X_train.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)
X_test = X_test.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)

In [15]:
randomForest(X_train, y_train, X_test, y_test)

0.22675306607146864

In [16]:
logit(X_train, y_train, X_test, y_test)

Optimization terminated successfully.
         Current function value: 0.623677
         Iterations 5


0.21519929870245788

Let's add a few more metrics using available data
* Home Shot Quantity = FTHG / HS
* Home Shot Quality = FTHG / HST
* Away Shot Quantity = FTAG / AS
* Away Shot Quality = FTAG / AST
* Home Win Percentage = $\frac{\sum I \cdot \text{homeWin}}{total games}$

Because these are fractions, there will be NaN values. If there are zero total shots and/or shots on target but a goal was scored for home/away respectively, we can interpret that as self goals. Therefore the shot quantity/quality percentage can be 0%

In [17]:
# Create Shot Quantity and Shot Quality columns for Home/Away
df['shotQntD_H'] = df.FTHG / df.HS
df['shotQntD_A'] = df.FTAG / df.AS
df['shotQltD_H'] = df.FTHG / df.HST
df['shotQltD_A'] = df.FTAG / df.AST

# Fill the percentages with 0 if Shots / Shots on Target are equal to zero
df = df.fillna(0)

In [18]:
# Replace the columns values with the difference of home/away
df['shotQntD_H'] = df.shotQntD_H - df.shotQntD_A
df['shotQntD_A'] = df.shotQntD_A - df.shotQntD_H
df['shotQltD_H'] = df.shotQltD_H - df.shotQltD_A
df['shotQltD_A'] = df.shotQltD_A - df.shotQltD_H

In [19]:
df_melt5 = pd.melt(df, id_vars='GameID', value_vars=['shotQntD_H', 'shotQntD_A'], var_name='isHome', value_name='sQntD')
df_melt5['isHome'] = np.where(df_melt5.isHome =='shotQntD_H', 'H', 'A')
df_merge = df_melt.merge(df_melt5, on=['GameID', 'isHome']).merge(df[['GameID', 'Date', 'Y']], on='GameID').sort_values('Date')
df_merge['hsQntD'] = df_merge.groupby(['Team', 'Y']).sQntD.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df_pivot = df_merge.pivot(index='GameID', columns='isHome')
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df[['GameID', 'Div', 'Y', 'HomeTeam', 'AwayTeam']].merge(df_pivot[['GameID', 'hsQntD_H', 'hsQntD_A']], on='GameID')
df = pd.merge(df, df_pivot[['GameID', 'hsQntD_H', 'hsQntD_A']], how='left')

In [20]:
df_melt6 = pd.melt(df, id_vars='GameID', value_vars=['shotQltD_H', 'shotQltD_A'], var_name='isHome', value_name='sQltD')
df_melt6['isHome'] = np.where(df_melt6.isHome =='shotQltD_H', 'H', 'A')
df_merge = df_melt.merge(df_melt6, on=['GameID', 'isHome']).merge(df[['GameID', 'Date', 'Y']], on='GameID').sort_values('Date')
df_merge['hsQltD'] = df_merge.groupby(['Team', 'Y']).sQltD.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df_pivot = df_merge.pivot(index='GameID', columns='isHome')
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]
df_pivot = df_pivot.reset_index()
df_pivot = df[['GameID', 'Div', 'Y', 'HomeTeam', 'AwayTeam']].merge(df_pivot[['GameID', 'hsQltD_H', 'hsQltD_A']], on='GameID')
df = pd.merge(df, df_pivot[['GameID', 'hsQltD_H', 'hsQltD_A']], how='left')

In [21]:
df_hs = df_hs.merge(df[['GameID', 'hsQntD_H', 'hsQntD_A', 'hsQltD_H', 'hsQltD_A']], how='left')
# Create Home Win Percentage Column
hw = df_hs.groupby(['HomeTeam', 'Y']).homeWin.transform(lambda x : x.expanding().sum().shift(1, fill_value = 0))
hg = df_hs.groupby(['HomeTeam', 'Y']).homeWin.transform(lambda x : x.expanding().count().shift(1, fill_value = 0))

df_hs['home_win_pct'] = hw / hg
df_hs = df_hs.fillna(0)
df_hs

,Div,Date,Y,HomeTeam,AwayTeam,GameID,hAGD_H,hAGD_A,homeWin,hxGD_H,hxGD_A,hST_H,hST_A,hsQntD_H,hsQntD_A,hsQltD_H,hsQltD_A,home_win_pct
0,Ligue_1,2014-08-08,14,Reims,Paris SG,5306,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ligue_1,2014-08-09,14,Nice,Toulouse,5313,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ligue_1,2014-08-09,14,Nantes,Lens,5312,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ligue_1,2014-08-09,14,Montpellier,Bordeaux,5311,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Ligue_1,2014-08-09,14,Bastia,Marseille,5307,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9125,Serie_A,2019-05-26,18,Fiorentina,Genoa,9080,0.259259,-0.222222,0,0.265487,-0.313340,1.189189,-1.081081,0.034188,0.019620,0.138975,0.093488,0.277778
9126,Serie_A,2019-05-26,18,Inter,Empoli,9081,0.555556,-0.430464,1,0.732493,-0.278262,1.945946,-1.810811,0.130969,0.037091,0.223951,0.027068,0.555556
9127,Serie_A,2019-05-26,18,Roma,Parma,9082,0.883598,-0.813333,1,0.404511,-0.628353,1.756757,-3.135135,0.092146,0.122316,0.136268,0.202230,0.611111
9128,Serie_A,2019-05-26,18,Sampdoria,Juventus,9083,-0.052910,1.359788,1,-0.034591,0.802564,0.783784,2.459459,0.146211,0.177717,0.199632,0.282999,0.500000


In [22]:
# Drop GameID and Y after train/test splits
X_train, y_train = df_hs[df.Y < 18].drop('homeWin', 1), df_hs[df.Y < 18].homeWin
X_test, y_test = df_hs[df.Y == 18].drop('homeWin', 1), df_hs[df.Y == 18].homeWin
X_train = X_train.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)
X_test = X_test.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)

In [23]:
lr_bsl = logit(X_train, y_train, X_test, y_test)

Optimization terminated successfully.
         Current function value: 0.623616
         Iterations 5


In [24]:
rf_bsl = randomForest(X_train, y_train, X_test, y_test)

Even after all the operations we went through, the random forest model without hyperparameter tuning does not perform better than the logit. From here, I will create a validation set using Y=17 from the training data and perform a Grid Search to optimize the hyperparameters in order to boost performance.

In [25]:
# Create validation splits along with training and testing
X_train, y_train = df_hs[df.Y < 17].drop('homeWin', 1), df_hs[df.Y < 17].homeWin
X_val, y_val = df_hs[df.Y == 17].drop('homeWin', 1), df_hs[df.Y == 17].homeWin
X_test, y_test = df_hs[df.Y == 18].drop('homeWin', 1), df_hs[df.Y == 18].homeWin

X_train = X_train.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)
X_val = X_val.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)
X_test = X_test.drop(['GameID', 'Y'], 1).select_dtypes(include=np.number)

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [100, 200, 500],
    'max_depth' : [1, 3, 5, 7]
}

rf_CV = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring = 'neg_brier_score', cv=4)
rf_CV.fit(X_val, y_val)
rf_CV.best_params_

{'max_depth': 5, 'n_estimators': 100}

In [27]:
rf = RandomForestClassifier(max_depth=5, n_estimators=200)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
rf_GS_bsl = brier_score_loss(y_test, y_pred)

# 2a. Your out-of-sample Brier score on 2018.



In [28]:
print('Brier Scores ')
print('\033[1mLogit:', lr_bsl)
print('\033[0mRandom Forest without Hyperparameter tuning:', rf_bsl)
print('Random Forest with Hyperparameter tuning:', rf_GS_bsl)

Brier Scores 
Logit: 0.21502632234832797
Random Forest without Hyperparameter tuning: 0.22116657988098606
Random Forest with Hyperparameter tuning: 0.21673665347107895


# 2b. The type of model you fit

I used **Logit** and **Random Forest Classifier**

# 2c. A very brief summary of the features used in your model. Be terse but precise so that the reader can figure out exactly how your features are computed.

Since much of the data is computed after the fact, I decided to use historical representations of the data based on the same stats just from previous games. I also decided to group these by year because stats do not usually translate from season to season.

I implemented historical representations of the following variables
* Goal Differential (hAGD)
* Shots on Target (hST)
* Expected Goals (hxG)

I also created new variables based on the given data and also transformed them into historical representations
* Shot Quantity (hsQntD)
* Shot Quality (hsQltD)
* Team Win Percentage (home_win_pct)



# 2d. A write-up of the process you used to build your model.